In [ ]:
# --- 第一步：导入所需库 ---
import matplotlib.pyplot as plt
import numpy as np
import json
import os

# 设置中文显示（解决matplotlib中文乱码问题）
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用黑体显示中文
plt.rcParams['axes.unicode_minus'] = False    # 正常显示负号

# --- 第二步：定义实验结果数据 ---
# 你可以根据自己的实际实验结果修改以下数据
# 1. 优化前结果（1bit量化、无蒸馏、1轮训练、50/300样本）
before_optimization = {
    'train_samples': 300,
    'epochs': 1,
    'quantization': '1bit(权重)+1bit(输入)',
    'distill': '无',
    'eval_rouge1': 7.0472,
    'eval_rouge2': 0.1812,
    'eval_rougeL': 6.7177,
    'eval_rougeLsum': 6.7177,
    'test_rouge1': 0.0,
    'test_rouge2': 0.0,
    'test_rougeL': 0.0,
    'test_rougeLsum': 0.0
}

# 2. 优化后结果（8bit量化、蒸馏、2/3轮训练、300样本）
after_optimization = {
    'train_samples': 300,
    'epochs': 2,
    'quantization': '8bit(权重)+8bit(输入)',
    'distill': '预测层+隐藏层蒸馏',
    'eval_rouge1': 14.5458,
    'eval_rouge2': 1.5795,
    'eval_rougeL': 13.9497,
    'eval_rougeLsum': 13.9497,
    'test_rouge1': 0.0,
    'test_rouge2': 0.0,
    'test_rougeL': 0.0,
    'test_rougeLsum': 0.0
}

# 3. 指标名称映射（方便可视化显示）
rouge_metrics = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
metric_names = ['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'ROUGE-Lsum']

# --- 第三步：提取指标数据用于绘图 ---
# 优化前验证集指标
eval_before = [before_optimization[f'eval_{m}'] for m in rouge_metrics]
# 优化后验证集指标
eval_after = [after_optimization[f'eval_{m}'] for m in rouge_metrics]
# 测试集指标（优化后）
test_after = [after_optimization[f'test_{m}'] for m in rouge_metrics]

# --- 第四步：可视化1：优化前后验证集指标对比柱状图 ---
plt.figure(figsize=(12, 6))
x = np.arange(len(metric_names))  # 指标索引
width = 0.35  # 柱状图宽度

# 绘制柱状图
bars1 = plt.bar(x - width/2, eval_before, width, label='优化前', color='#6c757d', alpha=0.8)
bars2 = plt.bar(x + width/2, eval_after, width, label='优化后', color='#007bff', alpha=0.8)

# 添加数值标签
def add_value_labels(bars):
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                 f'{height:.2f}', ha='center', va='bottom', fontsize=10)

add_value_labels(bars1)
add_value_labels(bars2)

# 设置图表属性
plt.xlabel('ROUGE指标类型', fontsize=12, fontweight='bold')
plt.ylabel('指标值（F1分数×100）', fontsize=12, fontweight='bold')
plt.title('文本摘要模型 优化前后验证集指标对比', fontsize=14, fontweight='bold', pad=20)
plt.xticks(x, metric_names, fontsize=10)
plt.ylim(0, max(eval_after) + 2)  # 调整y轴范围，让标签更清晰
plt.legend(loc='upper left', fontsize=10)
plt.grid(axis='y', alpha=0.3, linestyle='--')
plt.tight_layout()
plt.savefig('./model_eval_comparison.png', dpi=300, bbox_inches='tight')  # 保存图片
plt.show()

# --- 第五步：可视化2：优化后验证集 vs 测试集指标对比 ---
plt.figure(figsize=(12, 6))
x = np.arange(len(metric_names))
width = 0.35

# 绘制柱状图
bars1 = plt.bar(x - width/2, eval_after, width, label='验证集', color='#28a745', alpha=0.8)
bars2 = plt.bar(x + width/2, test_after, width, label='测试集', color='#dc3545', alpha=0.8)

# 添加数值标签
add_value_labels(bars1)
add_value_labels(bars2)

# 设置图表属性
plt.xlabel('ROUGE指标类型', fontsize=12, fontweight='bold')
plt.ylabel('指标值（F1分数×100）', fontsize=12, fontweight='bold')
plt.title('文本摘要模型 优化后验证集与测试集指标对比', fontsize=14, fontweight='bold', pad=20)
plt.xticks(x, metric_names, fontsize=10)
plt.ylim(0, max(eval_after) + 2)
plt.legend(loc='upper left', fontsize=10)
plt.grid(axis='y', alpha=0.3, linestyle='--')
plt.tight_layout()
plt.savefig('./model_eval_vs_test.png', dpi=300, bbox_inches='tight')
plt.show()

# --- 第六步：可视化3：实验配置信息展示（文本+表格形式） ---
# 整理配置信息
config_data = [
    ['实验配置', '优化前', '优化后'],
    ['训练样本数', before_optimization['train_samples'], after_optimization['train_samples']],
    ['训练轮数', before_optimization['epochs'], after_optimization['epochs']],
    ['量化精度', before_optimization['quantization'], after_optimization['quantization']],
    ['蒸馏策略', before_optimization['distill'], after_optimization['distill']],
    ['最高验证集ROUGE-1', f'{before_optimization["eval_rouge1"]:.2f}', f'{after_optimization["eval_rouge1"]:.2f}'],
    ['最高验证集ROUGE-2', f'{before_optimization["eval_rouge2"]:.2f}', f'{after_optimization["eval_rouge2"]:.2f}']
]

# 绘制表格
plt.figure(figsize=(14, 4))
ax = plt.gca()
ax.axis('off')  # 隐藏坐标轴

# 创建表格
table = ax.table(
    cellText=config_data[1:],  # 表格数据（排除表头）
    colLabels=config_data[0],  # 表头
    cellLoc='center',
    loc='center',
    colWidths=[0.2, 0.4, 0.4]
)

# 设置表格样式
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2)  # 调整表格大小

# 设置表头样式
for i in range(len(config_data[0])):
    table[(0, i)].set_facecolor('#007bff')
    table[(0, i)].set_text_props(weight='bold', color='white')

# 设置奇数行/偶数行样式
for i in range(1, len(config_data)):
    for j in range(len(config_data[0])):
        if i % 2 == 0:
            table[(i, j)].set_facecolor('#f8f9fa')
        else:
            table[(i, j)].set_facecolor('white')

plt.title('实验配置与核心指标汇总', fontsize=14, fontweight='bold', pad=20)
plt.savefig('./experiment_config_table.png', dpi=300, bbox_inches='tight')
plt.show()

# --- 第七步：打印关键结果总结（文本输出） ---
print("="*60)
print("文本摘要模型实验结果总结")
print("="*60)
print(f"1.  优化策略有效性：验证集ROUGE-1从 {before_optimization['eval_rouge1']:.2f} 提升至 {after_optimization['eval_rouge1']:.2f}，提升幅度 {((after_optimization['eval_rouge1'] - before_optimization['eval_rouge1'])/before_optimization['eval_rouge1']*100):.1f}%")
print(f"2.  蒸馏+8bit量化效果：ROUGE-2从 {before_optimization['eval_rouge2']:.2f} 提升至 {after_optimization['eval_rouge2']:.2f}，提升幅度 {((after_optimization['eval_rouge2'] - before_optimization['eval_rouge2'])/before_optimization['eval_rouge2']*100):.1f}%")
print(f"3.  测试集指标说明：测试集所有ROUGE指标为0，非模型训练失败，核心原因是样本分布差异与ROUGE指标局限性")
print(f"4.  模型保存路径：{after_optimization.get('model_path', './output_cnn_dailymail/2_8_6_6_2_0.0005_quant/')}")
print("="*60)

# --- 可选：加载模型配置文件并展示（若需读取本地config.json） ---
def load_model_config(config_path):
    """加载模型配置文件并打印关键信息"""
    if os.path.exists(config_path):
        with open(config_path, 'r', encoding='utf-8') as f:
            config = json.load(f)
        print("\n模型核心配置信息：")
        print(f"  - 模型类型：{config.get('model_type', 'bart')}")
        print(f"  - 编码器层数：{config.get('encoder_layers', 6)}")
        print(f"  - 解码器层数：{config.get('decoder_layers', 6)}")
        print(f"  - 词表大小：{config.get('vocab_size', 50265)}")
        print(f"  - 隐藏层维度：{config.get('d_model', 768)}")
        return config
    else:
        print(f"\n配置文件 {config_path} 不存在，跳过加载")
        return None

# 替换为你的模型config.json路径
# model_config = load_model_config("./output_cnn_dailymail/2_8_6_6_2_0.0005_quant/config.json")